## Домашнее задание

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

In [257]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [259]:
import pandas as pd
import torch
from torch import nn
from torchinfo import summary
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

In [260]:
df_train = pd.read_csv('train.csv', index_col='id')
df_train

,text,class
id,,
0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1
...,...,...
181462,Классным будет новый год..( http://t.co/LejaGu...,0
181463,Видишь человека хочешь сказать ему привет . А ...,0
181464,@Julia6_9Styles там длинный диагноз. Вкратце: ...,0


In [261]:
df_train['class'].unique()

array([0, 1], dtype=int64)

In [262]:
df_val = pd.read_csv('val.csv', index_col='id')
df_val

,text,class
id,,
181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
181471,"Хочу сделать как лучше, а получаю как всегда. :(",0
...,...,...
204145,А я знаю из-за кого такая паршивая погода была...,1
204146,ааа.... что с Гмейлом. не ужто он умер сегодня...,0
204147,"в учебники химии написано ""Пруст Жозеф Луи""\nя...",1


In [263]:
df_train['class'].unique()

array([0, 1], dtype=int64)

In [281]:
model_name = 'philschmid/distilbert-base-multilingual-cased-sentiment-2'
model = pipeline('text-classification', model=model_name, device=0)

In [282]:
# Проверка работы модели
model("На улице замечательная погода")

[{'label': 'positive', 'score': 0.46490734815597534}]

In [269]:
def make_prediction(text):
    pred = model(text)[0]['label']
    if pred == 'positive':
        pred = 1
    if pred == 'negative':
        pred = 0
        return pred

In [270]:
tqdm.pandas()

predictions = df_val['text'].progress_apply(lambda x: make_prediction(x))

  0%|                                                                                        | 0/22683 [00:00<?, ?it/s]C:\Users\Relict\AppData\Roaming\Python\Python39\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████| 22683/22683 [02:56<00:00, 128.61it/s]


In [271]:
(predictions == df_val['class']).sum() / len(df_val)

0.41498038178371466

In [272]:
# Текст №10 из тестового датасета

idx = 10
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
model(df_train.iloc[idx]['text'])

@LeraCookie вот красава, я в тебе не сомневался! ;)
label is 1


[{'label': 'positive', 'score': 0.7844205498695374}]

In [273]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [274]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._txts = [self.tokenizer(text, padding='max_length', max_length=70,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [275]:
torch.random.manual_seed(10)

y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=32,
                          shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=32,
                          shuffle=False)

In [276]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [277]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at philschmid/distilbert-base-multilingual-cased-sentiment-2 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [278]:
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              91,812,096
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 135,326,210
Trainable params: 135,326,210
Non-trainable params: 0

In [279]:
torch.random.manual_seed(10)
torch.backends.cudnn.deterministic = True

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
model = model.to(device)

param_groups = [
    {'params': model.pre_classifier.parameters(), 'lr': 0.001},
    {'params': model.classifier.parameters(), 'lr': 0.001}
]
optimizer = torch.optim.Adam(param_groups)

for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    for train_input, train_label in tqdm(train_loader):
        model.train()
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_ids=input_id, attention_mask=mask, labels=train_label)
                
        batch_loss = output.loss
        total_loss_train += batch_loss
        
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
        acc = (output.logits.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    with torch.no_grad():
        for val_input, val_label in valid_loader:
            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].squeeze(1).to(device)

            output = model(input_ids=input_id, attention_mask=mask, labels=val_label)

            batch_loss = output.loss
            total_loss_val += batch_loss

            acc = (output.logits.argmax(dim=1) == val_label).sum().item()
            total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_loader): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_loader): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at philschmid/distilbert-base-multilingual-cased-sentiment-2 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████████████████████████████████████████████████████████████████████████| 5671/5671 [04:49<00:00, 19.56it/s]


Epochs: 1 | Train Loss:  0.420         | Train Accuracy:  0.807         | Val Loss:  0.377         | Val Accuracy:  0.831


100%|██████████████████████████████████████████████████████████████████████████████| 5671/5671 [04:47<00:00, 19.73it/s]


Epochs: 2 | Train Loss:  0.407         | Train Accuracy:  0.814         | Val Loss:  0.372         | Val Accuracy:  0.838


In [ ]:
Вывод: Точность Val Accuracy после дообучения возросла до 83,8% против 41,5% изначальных валидационных данных.

## Дополнительные материалы:
 Фреймворк для упрощения обучения моделей (аналог keras): https://github.com/Lightning-AI/lightning

Аугментации : https://albumentations.ai/docs/api_reference/pytorch/transforms/

Метрики : https://github.com/Lightning-AI/torchmetrics

ИНСТРУКЦИИ ПО ПОТЕРЯМ : https://neptune.ai/blog/pytorch-loss-functions

Подбор гиперпараметров Optuna (Pytorch) : https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_simple.py

Optuna общий гайд : https://optuna.readthedocs.io/en/stable/tutorial/index.html